# Casco Bay FVCOM Data Acquisition and Re-grid

This is the data acquisition loop for downloading and preparing a subset of the FVCOM GOM data specific to the casco bay area. This notebook will chart the data acquisition steps used to download and regrid the FVCOM archival data for **2016-2023**.

In [1]:
# Libraries

# Import Libraries
import xarray as xr
import os
import matplotlib.pyplot as plt
import math
import numpy as np
import xesmf as xe
import regionmask

#importing modules used as auxiliary
from dateutil import parser
from datetime import datetime,timedelta
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union

# Set directory of where to save a manageable piece (relative path)
os.chdir("../data")
os.getcwd()

'/Users/akemberling/Documents/Repositories/FVCOM_indicators/data'

## Set 0: Load Data and Assign Coords

### Create List of URL Endpoints

The server's and my computer aren't going to like opening all the data at once, so the plan is to march through the months:

### Generate Base URL List

The folloring cell will generate the URLs to access monthly FVCOM data spanning a range of years and months. These will each be gigantic (90GB) and will not be be accessible beyond very limited functionality without further filtering steps covered below.

In [2]:
# Create a URL list that we can iterate through:
# hopefully it doesn't fail on us in the middle and ruin everything

# Start Year
start_yr = 2016

# End Year
end_yr = 2016

# Basic Structures for the year and month components
all_months = np.arange(1,13)
all_years  = np.arange(2016, 2023+1)

# Link base structure
base_url = "http://www.smast.umassd.edu:8080/thredds/dodsC/models/fvcom/NECOFS/Archive/NECOFS_GOM/"

# Generate all the URLs with a good old fashioned loop:
all_urls = []
for yr in all_years:
    yr_base = f"{base_url}{yr}/gom4_{yr}"
    
    for mon in all_months:
        yr_mon_ext = f"{yr_base}{mon:02d}.nc"
        all_urls.append(yr_mon_ext)


# Peak at the first few URLS
all_urls[0:2]

['http://www.smast.umassd.edu:8080/thredds/dodsC/models/fvcom/NECOFS/Archive/NECOFS_GOM/2016/gom4_201601.nc',
 'http://www.smast.umassd.edu:8080/thredds/dodsC/models/fvcom/NECOFS/Archive/NECOFS_GOM/2016/gom4_201602.nc']

## Flagging Lon/Lat Indices in BBOX

Take one day of data, identify the index numbers that fall within the domain we care about for Casco Bay. 

Our base domain is within the following bounding box:


lon_min = -70.4454   
lon_max = -69.8368   
lat_min = 43.5606   
lat_max = 43.9235   

We can't use ds.sel on irregular grid coordinates, but should be able to do a more standard masking routine or alter the links themselves to prevent ourselves from asking for too much un-needed data.

In [3]:
# Load one day:
# Just need date and coords
day_ds = xr.open_dataset(f"{all_urls[0]}?lon[0:1:53086],lat[0:1:53086],lonc[0:1:99136],latc[0:1:99136],time[0:1:1]")
day_ds

<xarray.Dataset>
Dimensions:  (node: 53087, nele: 99137, time: 2)
Coordinates:
  * time     (time) datetime64[ns] 2016-01-01 2016-01-01T01:01:52.499999744
Dimensions without coordinates: node, nele
Data variables:
    lon      (node) float32 ...
    lat      (node) float32 ...
    lonc     (nele) float32 ...
    latc     (nele) float32 ...
Attributes: (12/24)
    title:                           FVCOM GOM NECOFS HINDCAST UPDATE
    institution:                     School for Marine Science and Technology
    source:                          FVCOM_3.0
    history:                         Tue Mar 29 14:06:16 2016: ncrcat -d time...
    references:                      http://fvcom.smast.umassd.edu, http://co...
    Conventions:                     CF-1.0
    ...                              ...
    DODS.dimName:                    DateStrLen
    DODS_EXTRA.Unlimited_Dimension:  time
    Times.DODS.strlen:               26
    Times.DODS.dimName:              DateStrLen
    file_date.DODS.strlen:           26
    file_date.DODS.dimName:          DateStrLen

In [4]:
# Base Casco Bounding Box:
lon_min = -70.4454   
lon_max = -69.8368   
lat_min = 43.5606   
lat_max = 43.9235

---

## Find relevant lat & lon coordinates to use for subsetting:

Use a single time step to create mask and generate the new grid. Should then be able to use those index numbers to control what data gets pulled over THREDDS.

Need some way to convert this to an index range that the URL can accept...
ex. lon[0:**start_idx**:**end_idx**]

Because these coordinates aren't sorted, we may need to be inclusive of nodes we'd otherwise wish to ignore in order to capture all the nodes between the start index and the end index.

In [5]:
####  Get the Node Indices that Fall in Bounding Box:  ####

# Find the cases where lon coordinates fall within longitude limits
lon_set1 = day_ds.lon > lon_min 
lon_set2 = day_ds.lon < lon_max
lon_true = lon_set1 & lon_set1
lon_indices = [i for i, x in enumerate(lon_true) if x]

# Find the cases where lon coordinates fall within longitude limits
lat_set1 = day_ds.lat > lat_min 
lat_set2 = day_ds.lat < lat_max
lat_true = lat_set1 & lat_set1
lat_indices = [i for i, x in enumerate(lat_true) if x]


####  Do it again for centroids:  ####

# Find the cases where lon coordinates fall within longitude limits
c_lon_set1 = day_ds.lonc > lon_min 
c_lon_set2 = day_ds.lonc < lon_max
c_lon_true = c_lon_set1 & c_lon_set1
c_lon_indices = [i for i, x in enumerate(c_lon_true) if x]


# Find the cases where lon coordinates fall within longitude limits
c_lat_set1 = day_ds.latc > lat_min 
c_lat_set2 = day_ds.latc < lat_max
c_lat_true = c_lat_set1 & c_lat_set1
c_lat_indices = [i for i, x in enumerate(c_lat_true) if x]


####  Get the indices that satisfy both lat and lon:  ####
# How many nodes meet both the lon + lat criteria
# Nodal coordinate indices
nodes_true = lon_true & lat_true
node_indices = [i for i, x in enumerate(nodes_true) if x]
zone_true = c_lon_true & c_lat_true
zone_indices = [i for i, x in enumerate(zone_true) if x]

print("Total nodes within bounding box:", len(node_indices))
print("Total centroids within bounding box:", len(zone_indices))

Total nodes within bounding box: 8718
Total centroids within bounding box: 14720


###  Why Index Range Doesn't Work Here

The opendap links can take a min/max index number to limit the amount of data requested. In this case this won't work for us because the lon & lat information isn't sorted. This means that using a min/max of the relevant indices ends up bringing along many nodes in the mesh that we don't want.

In [6]:
# Get the min and max, very crude way to get a start and stop that we could use with the opendap link...
print(f"Latitude index min/max: lat[0:{min(lat_indices)+1}:{max(lat_indices)+1}]")
print(f"Longitude index min/max: lon[0:{min(lon_indices)+1}:{max(lon_indices)+1}]")

# Get the min and max, very crude way to get a start and stop that we could use with the opendap link...
print(f"Zonal Latitude index min/max: lat[0:{min(c_lat_indices)+1}:{max(c_lat_indices)+1}]")
print(f"Zonal Longitude index min/max: lon[0:{min(c_lon_indices)+1}:{max(c_lon_indices)+1}]")

# Seems to still want more or less all the data...

Latitude index min/max: lat[0:1:52935]
Longitude index min/max: lon[0:1:52935]
Zonal Latitude index min/max: lat[0:1:98907]
Zonal Longitude index min/max: lon[0:1:98907]


---

## Load Essential Set of Variables

We are compute and memory limited here, so gotta leave what we don't need behind for performance

We need:
 - **lon**:      Nodal longitude
 - **lat**:      Nodal latitude
 - **lonc**:     Zonal longitude
 - **lat**:      Zonal latitude
 - **time**:     Time
 - **temp**:     Sea water temperature
 - **salinity**: Sea water salinity
 - **ua**:       Vertically Averaged x-velocity
 - **va**:       Vertically Averaged y-velocity
 - **u**:        Eastward Water Velocity
 - **v**:        Northward Water Velocity
 - **ww**:       Upward Water Velocity



###  Filtering Extensions with URLs

Server-side data filters can be applied through modification of the URL links. Each THREDDS catalog entry has a link to its relevant data access form, that will generate the appropriate URL structures based on data needs.

ex. http://www.smast.umassd.edu:8080/thredds/dodsC/models/fvcom/NECOFS/Archive/NECOFS_GOM/2016/gom4_201601.nc.html

For the 12 coordinates/variables needed, we will be altering the following text which is an extension of the base URLs:

**BASE_URL**?lon[0:1:53086],lat[0:1:53086],lonc[0:1:99136],latc[0:1:99136],time[0:1:743],u[0:1:743][0:1:39][0:1:99136],v[0:1:743][0:1:39][0:1:99136],ww[0:1:743][0:1:39][0:1:99136],ua[0:1:743][0:1:99136],va[0:1:743][0:1:99136],temp[0:1:743][0:1:39][0:1:53086],salinity[0:1:743][0:1:39][0:1:53086]


## Step 1: Load Data and Assign Coords

In [7]:
def open_and_prep_fvcom(
        archive_url,
        keep_variables,
        drop_variables = ['siglay','siglev']):
    """Lazy-Load an FVCOM Archive Month, Fix Coords + Dates"""
    
    # A. Load a URL
    # lazy load of the data
    ds = xr.open_dataset(archive_url, drop_variables = drop_variables, decode_times = False).chunk({"time" : 10})

    # convert lon/c, lat/c to coordinates
    ds = ds.assign_coords({var:ds[var] for var in ['lon','lat','lonc','latc']})
    
    # B. Fix the Dates:
    # the first day
    dt0 = parser.parse(ds.time.attrs['units'].replace('days since ',''))

    # parse dates summing days to the origin
    ds = ds.assign(time = [dt0 + timedelta(seconds = day * 86400) for day in ds.time.values])

    # Subset down to main variables
    ds = ds[keep_variables]
    
    return ds

## Loading a Test Month:

At this point the goal is to see if THREDDS/OpenDap will be permissive enough to `ds.load()` one month of data for the node and centroid indices we actually need. I will also try to only load the essential variables as well so that its the smallest complete set of information.

In [8]:
# Lazy Load and fix the coordinates and dates

# This is the core list
variables = ['temp', 'salinity', 'nv', 'ua', 'va', 'u', 'v', 'ww']

# Load one of the URLs - This could be looped
ds = open_and_prep_fvcom(all_urls[0], keep_variables = variables, drop_variables=['siglay','siglev'])
ds

<xarray.Dataset>
Dimensions:   (time: 744, siglay: 40, node: 53087, three: 3, nele: 99137)
Coordinates:
    x         (node) float32 dask.array<chunksize=(53087,), meta=np.ndarray>
    y         (node) float32 dask.array<chunksize=(53087,), meta=np.ndarray>
    lon       (node) float32 dask.array<chunksize=(53087,), meta=np.ndarray>
    lat       (node) float32 dask.array<chunksize=(53087,), meta=np.ndarray>
    lonc      (nele) float32 dask.array<chunksize=(99137,), meta=np.ndarray>
    latc      (nele) float32 dask.array<chunksize=(99137,), meta=np.ndarray>
  * time      (time) datetime64[ns] 2016-01-01 ... 2016-01-31T22:58:07.500000
Dimensions without coordinates: siglay, node, three, nele
Data variables:
    temp      (time, siglay, node) float32 dask.array<chunksize=(10, 40, 53087), meta=np.ndarray>
    salinity  (time, siglay, node) float32 dask.array<chunksize=(10, 40, 53087), meta=np.ndarray>
    nv        (three, nele) int32 dask.array<chunksize=(3, 99137), meta=np.ndarray>
    ua        (time, nele) float32 dask.array<chunksize=(10, 99137), meta=np.ndarray>
    va        (time, nele) float32 dask.array<chunksize=(10, 99137), meta=np.ndarray>
    u         (time, siglay, nele) float32 dask.array<chunksize=(10, 40, 99137), meta=np.ndarray>
    v         (time, siglay, nele) float32 dask.array<chunksize=(10, 40, 99137), meta=np.ndarray>
    ww        (time, siglay, nele) float32 dask.array<chunksize=(10, 40, 99137), meta=np.ndarray>
Attributes: (12/20)
    title:                           FVCOM GOM NECOFS HINDCAST UPDATE
    institution:                     School for Marine Science and Technology
    source:                          FVCOM_3.0
    history:                         Tue Mar 29 14:06:16 2016: ncrcat -d time...
    references:                      http://fvcom.smast.umassd.edu, http://co...
    Conventions:                     CF-1.0
    ...                              ...
    Icing_Model_Forcing:             FVCOM variable surface icing forcing:\nF...
    Special_Physical_processes:      long shore flow adjustment for thermal w...
    nco_openmp_thread_number:        1
    DODS.strlen:                     26
    DODS.dimName:                    DateStrLen
    DODS_EXTRA.Unlimited_Dimension:  time

In [9]:
# Grab the variables we need
# From only the coordinates we need:
ds = ds.isel(node = node_indices, nele = zone_indices)
ds

/Users/akemberling/opt/anaconda3/envs/py3oisst/lib/python3.8/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


<xarray.Dataset>
Dimensions:   (time: 744, siglay: 40, node: 8718, three: 3, nele: 14720)
Coordinates:
    x         (node) float32 dask.array<chunksize=(8718,), meta=np.ndarray>
    y         (node) float32 dask.array<chunksize=(8718,), meta=np.ndarray>
    lon       (node) float32 dask.array<chunksize=(8718,), meta=np.ndarray>
    lat       (node) float32 dask.array<chunksize=(8718,), meta=np.ndarray>
    lonc      (nele) float32 dask.array<chunksize=(14720,), meta=np.ndarray>
    latc      (nele) float32 dask.array<chunksize=(14720,), meta=np.ndarray>
  * time      (time) datetime64[ns] 2016-01-01 ... 2016-01-31T22:58:07.500000
Dimensions without coordinates: siglay, node, three, nele
Data variables:
    temp      (time, siglay, node) float32 dask.array<chunksize=(10, 40, 8718), meta=np.ndarray>
    salinity  (time, siglay, node) float32 dask.array<chunksize=(10, 40, 8718), meta=np.ndarray>
    nv        (three, nele) int32 dask.array<chunksize=(3, 14720), meta=np.ndarray>
    ua        (time, nele) float32 dask.array<chunksize=(10, 14720), meta=np.ndarray>
    va        (time, nele) float32 dask.array<chunksize=(10, 14720), meta=np.ndarray>
    u         (time, siglay, nele) float32 dask.array<chunksize=(10, 40, 14720), meta=np.ndarray>
    v         (time, siglay, nele) float32 dask.array<chunksize=(10, 40, 14720), meta=np.ndarray>
    ww        (time, siglay, nele) float32 dask.array<chunksize=(10, 40, 14720), meta=np.ndarray>
Attributes: (12/20)
    title:                           FVCOM GOM NECOFS HINDCAST UPDATE
    institution:                     School for Marine Science and Technology
    source:                          FVCOM_3.0
    history:                         Tue Mar 29 14:06:16 2016: ncrcat -d time...
    references:                      http://fvcom.smast.umassd.edu, http://co...
    Conventions:                     CF-1.0
    ...                              ...
    Icing_Model_Forcing:             FVCOM variable surface icing forcing:\nF...
    Special_Physical_processes:      long shore flow adjustment for thermal w...
    nco_openmp_thread_number:        1
    DODS.strlen:                     26
    DODS.dimName:                    DateStrLen
    DODS_EXTRA.Unlimited_Dimension:  time

### Plot Check

Always good to take a look at a map and see what the data looks like.

In [11]:
# See if a single variable will load()
single_var_ds = ds[["nv", "temp"]].load()
single_var_ds

<xarray.Dataset>
Dimensions:  (three: 3, nele: 14720, time: 744, siglay: 40, node: 8718)
Coordinates:
    x        (node) float32 1.7e+06 1.705e+06 1.71e+06 ... 1.218e+06 1.218e+06
    y        (node) float32 4.205e+05 4.149e+05 ... 3.011e+05 3.014e+05
    lon      (node) float32 -59.81 -59.75 -59.7 -59.64 ... -66.1 -66.1 -66.1
    lat      (node) float32 46.15 46.09 46.04 45.98 ... 45.48 45.47 45.47 45.47
    lonc     (nele) float32 -59.84 -59.8 -59.76 -59.77 ... -66.1 -66.1 -66.11
    latc     (nele) float32 46.1 46.1 46.06 46.03 ... 45.47 45.47 45.47 45.47
  * time     (time) datetime64[ns] 2016-01-01 ... 2016-01-31T22:58:07.500000
Dimensions without coordinates: three, nele, siglay, node
Data variables:
    nv       (three, nele) int32 96 95 3 95 97 ... 52924 52934 52926 52926 52935
    temp     (time, siglay, node) float32 3.422 3.433 3.245 ... 4.999 4.996 5.0
Attributes: (12/20)
    title:                           FVCOM GOM NECOFS HINDCAST UPDATE
    institution:                     School for Marine Science and Technology
    source:                          FVCOM_3.0
    history:                         Tue Mar 29 14:06:16 2016: ncrcat -d time...
    references:                      http://fvcom.smast.umassd.edu, http://co...
    Conventions:                     CF-1.0
    ...                              ...
    Icing_Model_Forcing:             FVCOM variable surface icing forcing:\nF...
    Special_Physical_processes:      long shore flow adjustment for thermal w...
    nco_openmp_thread_number:        1
    DODS.strlen:                     26
    DODS.dimName:                    DateStrLen
    DODS_EXTRA.Unlimited_Dimension:  time

In [52]:
single_var_ds.temp

<xarray.DataArray 'temp' (time: 744, siglay: 40, node: 8718)>
array([[[3.421954  , 3.4334905 , 3.2449572 , ..., 7.8173685 ,
         7.8206477 , 8.042814  ],
        [1.5112784 , 1.6019442 , 1.2881873 , ..., 7.827018  ,
         7.8206477 , 8.042814  ],
        [1.5044409 , 1.6002367 , 1.2900745 , ..., 7.835831  ,
         7.8206477 , 8.042814  ],
        ...,
        [1.4675883 , 1.5331064 , 1.3104594 , ..., 8.212206  ,
         8.216722  , 8.215397  ],
        [1.4700543 , 1.5337615 , 1.3118784 , ..., 8.213808  ,
         8.217655  , 8.216392  ],
        [1.4735585 , 1.53444   , 1.3136157 , ..., 8.214601  ,
         8.218123  , 8.216874  ]],

       [[4.25828   , 4.356971  , 4.3751082 , ..., 7.7911005 ,
         7.778472  , 8.028116  ],
        [1.3680563 , 1.430532  , 1.2069913 , ..., 7.7911005 ,
         7.778472  , 8.028116  ],
        [1.3646007 , 1.4295979 , 1.2091961 , ..., 7.801796  ,
         7.7919784 , 8.028116  ],
...
        [0.8370356 , 1.0954424 , 0.6347279 , ..., 4.9983196 ,
         4.994594  , 4.9997888 ],
        [0.8373164 , 1.0952576 , 0.63393945, ..., 4.998872  ,
         4.995353  , 5.0003595 ],
        [0.8380647 , 1.095149  , 0.633439  , ..., 4.9991536 ,
         4.9957447 , 5.000652  ]],

       [[1.6459724 , 1.6394675 , 1.4382409 , ..., 4.871354  ,
         4.867725  , 4.8722215 ],
        [0.9796338 , 1.0573077 , 0.63694876, ..., 4.8776865 ,
         4.8728094 , 4.8783474 ],
        [0.9711736 , 1.0557853 , 0.6374307 , ..., 4.884575  ,
         4.8782454 , 4.8850203 ],
        ...,
        [0.7441311 , 0.91549224, 0.5814597 , ..., 4.998454  ,
         4.9952483 , 4.999599  ],
        [0.74532485, 0.91547763, 0.58160394, ..., 4.9989486 ,
         4.9959764 , 5.000124  ],
        [0.7468393 , 0.915653  , 0.5818047 , ..., 4.999198  ,
         4.996345  , 5.0003905 ]]], dtype=float32)
Coordinates:
    x        (node) float32 1.7e+06 1.705e+06 1.71e+06 ... 1.218e+06 1.218e+06
    y        (node) float32 4.205e+05 4.149e+05 ... 3.011e+05 3.014e+05
    lon      (node) float32 -59.81 -59.75 -59.7 -59.64 ... -66.1 -66.1 -66.1
    lat      (node) float32 46.15 46.09 46.04 45.98 ... 45.48 45.47 45.47 45.47
  * time     (time) datetime64[ns] 2016-01-01 ... 2016-01-31T22:58:07.500000
Dimensions without coordinates: siglay, node
Attributes:
    long_name:      temperature
    standard_name:  sea_water_temperature
    units:          degrees_C
    grid:           fvcom_grid
    type:           data

In [72]:
# What if we just started from scratch - one way for the nodes, another way for the centroids


# This is how I need temperature to look to add it as a variable into a dataset that has swapped lon + lat dimensions in place of node

xr.DataArray(
    data = single_var_ds.temp.values, 
    dims = ["time", "lon", "lat", "depth"],
    coords= dict(
        lon = single_var_ds.lon.values,
        lat = single_var_ds.lat.values,
        time = single_var_ds.time.values,
        depth =  single_var_ds.siglay.values
    ))

# I hate you nodes...


ValueError: different number of dimensions on data and dims: 3 vs 4

In [65]:
single_var_ds

<xarray.Dataset>
Dimensions:  (three: 3, nele: 14720, time: 744, siglay: 40, node: 8718)
Coordinates:
    x        (node) float32 1.7e+06 1.705e+06 1.71e+06 ... 1.218e+06 1.218e+06
    y        (node) float32 4.205e+05 4.149e+05 ... 3.011e+05 3.014e+05
    lon      (node) float32 -59.81 -59.75 -59.7 -59.64 ... -66.1 -66.1 -66.1
    lat      (node) float32 46.15 46.09 46.04 45.98 ... 45.48 45.47 45.47 45.47
    lonc     (nele) float32 -59.84 -59.8 -59.76 -59.77 ... -66.1 -66.1 -66.11
    latc     (nele) float32 46.1 46.1 46.06 46.03 ... 45.47 45.47 45.47 45.47
  * time     (time) datetime64[ns] 2016-01-01 ... 2016-01-31T22:58:07.500000
Dimensions without coordinates: three, nele, siglay, node
Data variables:
    nv       (three, nele) int32 96 95 3 95 97 ... 52924 52934 52926 52926 52935
    temp     (time, siglay, node) float32 3.422 3.433 3.245 ... 4.999 4.996 5.0
Attributes: (12/20)
    title:                           FVCOM GOM NECOFS HINDCAST UPDATE
    institution:                     School for Marine Science and Technology
    source:                          FVCOM_3.0
    history:                         Tue Mar 29 14:06:16 2016: ncrcat -d time...
    references:                      http://fvcom.smast.umassd.edu, http://co...
    Conventions:                     CF-1.0
    ...                              ...
    Icing_Model_Forcing:             FVCOM variable surface icing forcing:\nF...
    Special_Physical_processes:      long shore flow adjustment for thermal w...
    nco_openmp_thread_number:        1
    DODS.strlen:                     26
    DODS.dimName:                    DateStrLen
    DODS_EXTRA.Unlimited_Dimension:  time

In [71]:
# These are the coordinates we'd like to see (except we lost depth)
new_svar = single_var_ds.drop(["lon", "lat"]).expand_dims(dim = {"lon" : single_var_ds.lon.values, "lat" : single_var_ds.lat.values}).drop_dims(["node", "nele", "three"])


# We lost all the variables that go with the node dimension, which shouldn't be surprising but is still annoying

# Add depth back
new_svar = new_svar.expand_dims(dim = {"siglay" : single_var_ds.siglay.values})
new_svar["temp"] = single_var_ds.temp.values

MissingDimensionsError: cannot set variable 'temp' with 3-dimensional data without explicit dimension names. Pass a tuple of (dims, data) instead.

In [16]:
# Size check - 1 GB
# single_var_ds.drop().to_netcdf("fvcom_firstmonth_temp.nc")

# years * months * variables
#7*12*8

# Should drop depths we don't need

---

## Regrid to Standard Rectangular Grid

This should permit familiar operations like lat/lon selection and which will let us dig into the data in more familiar+conventional means. 

This involves two steps:
 1. Creation of a regular rectangular grid to interpolate to
 2. Re-gridding to said grid

**ALL** of the following functions are from Alex Kerney and NERACOOS.
https://github.com/gulfofmaine/NERACOOS_ERDDAP_K8S/blob/main/datasets/UMass/FVCOM/gom3_common.py 


In [17]:
def latest_time(ds: xr.Dataset):
    """Extracts the latest time from the dataset

    Returns a Python datetime and the integer timestamp
    """
    latest_timestamp = ds["time"].max().item()
    dt = pd.to_datetime(latest_timestamp).to_pydatetime()

    return (dt, latest_timestamp)


def round_up(n: float, decimals: int = 0):
    """Round up by a specified number of decimals"""
    multiplier = 10**decimals
    return math.ceil(n * multiplier) / multiplier


def round_down(n: float, decimals: int = 0):
    """Round down by a specified number of decimal places"""
    multiplier = 10**decimals
    return math.floor(n * multiplier) / multiplier

In [18]:
def grid_ds(ds: xr.Dataset,
            grid_spacing: float,
            rounding_decimals: float):
    """Generate a regular grid for the input dataset"""
    
    ds_grid = xe.util.grid_2d(
        round_down(ds["lon"].min(), rounding_decimals),
        round_up(ds["lon"].max(), rounding_decimals),
        grid_spacing,
        round_down(ds["lat"].min(), rounding_decimals),
        round_up(ds["lat"].max(), rounding_decimals),
        grid_spacing,
    )


    return ds_grid


In [19]:
def mask_ds(
    ds: xr.Dataset,
    ds_grid: xr.Dataset):
    """Generate dataset mask, and use cached version if possible"""

    node_ds = ds[["lat", "lon", "nv"]]
    node_ds = node_ds.load()
    node_triangles = []

    for nele_index in range(node_ds.dims["nele"]):
        points = []

        if nele_index % 1000 == 0:
            print(
                f"Processing mask triangles {nele_index}/{node_ds.dims['nele']}",
            )

        for node in node_ds["nv"].isel(nele=nele_index):
            node_index = node.values
            point_ds = node_ds.isel(node=node_index - 1)
            point = Point(point_ds["lon"].item(), point_ds["lat"].item())
            points.append(point)

        tri = Polygon(points)
        node_triangles.append(tri)

    mask_polygon = unary_union(node_triangles)
    mask = regionmask.Regions([mask_polygon])
    ds_grid["mask"] = mask.mask(ds_grid) == 0

    return ds_grid

In [20]:
def clone_dataset_attributes(target_ds: xr.Dataset, source_ds: xr.Dataset):
    """Clone most global and variable attributes from the source dataset to the target"""
    global_skip_keys = {"CoordinateSystem", "CoordinateProjection"}
    variable_skip_keys = {"type", "mesh", "location"}

    for key, value in source_ds.attrs.items():
        if key not in global_skip_keys:
            target_ds.attrs[key] = value

    for var in target_ds.variables:
        try:
            for key, value in source_ds[var].attrs.items():
                if key not in variable_skip_keys:
                    target_ds[var].attrs[key] = value
        except KeyError:
            pass


In [21]:
def regrid_ds(
    ds: xr.Dataset,
    ds_grid_masked: xr.Dataset):
    """Regrid FVCOM data dynamically from OpenDAP"""

    # load from OpenDap
    ds = ds.load()

    # Generate regridder
    regridder = xe.Regridder(ds, ds_grid_masked, "nearest_s2d", locstream_in=True)

    # Perform regridding
    ds_regridded = regridder(ds)
    ds_regridded = ds_regridded.where(ds_grid_masked["mask"])

    # Format Dims
    ds_regridded["lat"] = ds_regridded["lat"].isel(x=0)
    ds_regridded["lon"] = ds_regridded["lon"].isel(y=0)
    ds_regridded = ds_regridded.swap_dims({"y": "lat", "x": "lon"})
    ds_regridded = ds_regridded.rename_dims({"lon": "longitude", "lat": "latitude"})
    ds_regridded = ds_regridded.rename({"lon": "longitude", "lat": "latitude"})
    # Bring in the attributes
    clone_dataset_attributes(ds_regridded, ds)
    return ds_regridded

## Step 1: Make Grid - Perform Once

This step only needs a single timestep of coordinate information, and then it can be recycled for future files.

In [22]:
fvcom_grid =  grid_ds(single_var_ds,grid_spacing = 0.5, rounding_decimals = 3)
fvcom_grid

<xarray.Dataset>
Dimensions:  (y: 6, x: 26, y_b: 7, x_b: 27)
Coordinates:
    lon      (y, x) float64 -70.12 -69.62 -69.12 -68.62 ... -58.62 -58.12 -57.62
    lat      (y, x) float64 43.81 43.81 43.81 43.81 ... 46.31 46.31 46.31 46.31
    lon_b    (y_b, x_b) float64 -70.37 -69.87 -69.37 ... -58.37 -57.87 -57.37
    lat_b    (y_b, x_b) float64 43.56 43.56 43.56 43.56 ... 46.56 46.56 46.56
Dimensions without coordinates: y, x, y_b, x_b
Data variables:
    *empty*

##  Step 2: Process Masking for Dataset Triangles - Perform Once

Each timestep and across all years and months the node locations will be the same. This means the masking step of identifying which nodes and centroids are where needs only to be done one time.

In [23]:
# Perform the Masking
fvcom_mask = mask_ds(ds = single_var_ds, ds_grid = fvcom_grid)
fvcom_mask

Processing mask triangles 0/14720
Processing mask triangles 1000/14720


IndexError: index 9465 is out of bounds for axis 0 with size 8718

## Step 3: Regrid - Perform for All

Each month will need to be regrid, just the way it goes.

**Note:** This step is where the data is loaded, and this means it will take a while

####  Step 3A.: Find Alternative Lat/Lon Filtering

In [23]:
# Can't use ds.sel on irregular grid coordinates, but should be able to do a more standard masking routine
lon_min = -70.4454
lon_max = -69.8368
lat_min = 43.5606
lat_max = 43.9235

In [24]:
# Masking using where
dumb_selection = ds.where(ds.lon > lon_min, drop = True)
dumb_selection

Error:curl error: Timeout was reached
curl error details: 


In [43]:
# # Now the fun part - nvm too big
# new_ds = regrid_ds(ds = ds, ds_grid_masked = fvcom_mask)
# new_ds

oc_open: server error retrieving url: code=403 message="Request too big=11801.0 Mbytes, max=1000.0"oc_open: server error retrieving url: code=403 message="Request too big=11801.0 Mbytes, max=1000.0"oc_open: server error retrieving url: code=403 message="Request too big=11801.0 Mbytes, max=1000.0"oc_open: server error retrieving url: code=403 message="Request too big=11801.0 Mbytes, max=1000.0"oc_open: server error retrieving url: code=403 message="Request too big=11801.0 Mbytes, max=1000.0"oc_open: server error retrieving url: code=403 message="Request too big=11801.0 Mbytes, max=1000.0"oc_open: server error retrieving url: code=403 message="Request too big=11801.0 Mbytes, max=1000.0"

RuntimeError: NetCDF: Authorization failure

## Step 4: Crop

## Step 5: Save

In [1]:
7386.44/15


492.4293333333333

In [2]:
7386.44/4

1846.61